# Explore PWT Data

## Setup

In [ ]:
from util.plot import plot_scatter_with_text

## Basic (Development) Income Accounting

Following Julieta Caunedo's lecture in STEG

Income accounting: $$\ln \left(\frac{Y}{L}\right)=\ln (Z)+\frac{\alpha}{1-\alpha} \ln \left(\frac{K}{Y}\right)+\ln \left(\frac{H}{L}\right)$$

### Setup data

In [ ]:
# read PWT10.0 data
df_PWT = pd.read_stata("https://www.rug.nl/ggdc/docs/pwt100.dta")

In [ ]:
# read WDI natural resource rents data from Julieta Caunedo's repo
url_NR = "https://github.com/julicaunedo/STEG_Lecture2/blob/main/natural_resources.dta?raw=true"
df_NR = pd.read_stata(url_NR)

In [ ]:
# merge two data
df = df_PWT.merge(df_NR, on=["countrycode", "year"], how="left")

In [ ]:
##relevant variables

# real PPP (in mil 2017US$)
# PWT capital corresponds to reproducible capital

# Output: rgdpo
# Human Capital (per worker): hc
# Labor Inputs/Workers: emp
# Average hours: avh
# Capital stock: cn


alpha = 1 / 3  # capital share H&Jones
year_base = 2017

# adjustment for natural resources on GDP.
# assuming one uses the same deflator for GDP and nat_res
df = df.eval(
    """
    rgdpo = (1-natural_res/100)*rgdpo
    cgdpo = (1-natural_res/100)*cgdpo
"""
)

# income accounting variables
# Jones uses employment measures, not average hours(were not available back then)
# Capital denominated at current PPP $ so capital-output ratio computed using cgdpo
df = df.eval(
    """
    output_per_worker = log(rgdpo/(emp))
    output_per_worker_hour = log(rgdpo/(emp*avh))
    output_per_worker_PPPc = log(cgdpo/(emp))
    output_per_worker_XRc = log(cgdpo*pl_gdpo/(emp))
    
    capital_output_ratio = log(cn/cgdpo)*@alpha/(1-@alpha)
    human_capital_per_worker = log(hc)

    tfp_resid = output_per_worker-capital_output_ratio-human_capital_per_worker

    capital_output_ratio_sh = log(cn/cgdpo)*(1-labsh)/(labsh)
    tfp_resid_labsh = output_per_worker-capital_output_ratio_sh-human_capital_per_worker
"""
)
# Can use Capital services instead of astocks is desired.
country_average_cn = df.groupby("year")["cn"].transform("mean")
df = df.eval(
    """
    capserv_output_ratio = log(@country_average_cn*ck/cgdpo)*@alpha/(1-@alpha)
"""
)

# normalized accounting variables, with USA=1
cols = [
    "output_per_worker",
    "output_per_worker_hour",
    "output_per_worker_PPPc",
    "output_per_worker_XRc",
    "capital_output_ratio",
    "capital_output_ratio_sh",
    "capserv_output_ratio",
    "human_capital_per_worker",
    "tfp_resid",
    "tfp_resid_labsh",
]
norm_values = (
    (
        pd.concat((df[["countrycode", "year"]], df[cols].apply(np.exp)), axis=1)
        .groupby("year")
        .apply(
            lambda x: x.set_index("countrycode")
            .drop("year", axis=1)
            .apply(lambda x: x / x["USA"])
        )
    )
    .rename(columns=lambda x: "norm_" + x)
    .reset_index()
)
df = df.merge(norm_values, on=["countrycode", "year"])

### Balassa-Samuelson effect

In [ ]:
df_temp = df.query('year==@year_base & countrycode in ["ARG", "CHN", "FIN", "USA"]')

In [ ]:
ax = (
    df_temp.set_index(["country", "year"])[
        ["output_per_worker_PPPc", "output_per_worker_XRc"]
    ]
    .rename_axis("var", axis=1).stack().rename("value").reset_index()
    .pipe(
        (sns.catplot, "data"), kind="bar",
        x="country", y="value", hue="var",
        alpha=0.7, 
    )
)
ax.set(xlabel="", 
       ylabel=f"Log Output per worker {year_base}, current 2017 US$ (USA=1)",
       ylim=(0,15)
      );

In [ ]:
def plot_scatter_with_text(
        data, x, y, z, hue=None, style=None,
        text_condition=None, text_adjust=False, adjust_precision=0.1):
    """
    Use seaborn, matplotlib and adjustText to plot scatter with text
    """
    data = data.copy().reset_index(drop=True)
    ax = sns.scatterplot(data=data, x=x, y=y, hue=hue, style=style)
    if isinstance(text_condition, pd.Series):
        data = data[text_condition].reset_index(drop=True)
    texts = [ax.annotate(text, (data.loc[i, x], data.loc[i, y])) for i, text in enumerate(data[z].values)]
    if text_adjust:
        adjust_text(texts, precision=adjust_precision)
    return ax

### Working hour and development

In [ ]:
df_temp = df.query('year==@year_base').reset_index(drop=True)

plot_scatter_with_text(df_temp,"output_per_worker","avh","countrycode",)

df.plot.scatter("output_per_worker", "avh");

### Income contributions

In [ ]:
df = df.eval(
    """
    overall_diff = 1/norm_output_per_worker
    contrib_capital = 1/norm_capital_output_ratio
    contrib_capital_sh = 1/norm_capital_output_ratio_sh
    contrib_capital_serv = 1/norm_capserv_output_ratio
    contrib_labor = 1/norm_human_capital_per_worker
    contrib_TFP = 1/norm_tfp_resid
    share_due_to_TFP = contrib_TFP/(contrib_TFP+(contrib_labor*contrib_capital))
    share_due_to_hcap=contrib_labor/(contrib_labor+contrib_capital)*(1-share_due_to_TFP)
    share_due_to_cap=contrib_capital/(contrib_capital+contrib_labor)*(1-share_due_to_TFP)
"""
)

In [ ]:
countries = ["USA","HKG","SGP","FRA","DEU","GBR","JPN","KOR","ARG","MEX",
"BWA","ZAF","BRA","THA","CHN","IDN","IND","KEN","MWI"] 
prints(df.query('year==@year_base & countrycode in @countries')[
    [
        "country",
        "norm_output_per_worker",
        "norm_capital_output_ratio",
        "norm_human_capital_per_worker",
        "norm_tfp_resid",
        "share_due_to_TFP",
    ]
].dropna().sort_values("norm_output_per_worker", ascending=False).reset_index(drop=True).set_axis(
    ["Country", "Y/L", "(K/Y)^(α/(1-α))", "H/L", "Z","share due to TFP"], axis=1
))

In [ ]:
ax = plot_scatter_with_text(
    df.query('year==@year_base'),
    "norm_output_per_worker_PPPc",
    "share_due_to_TFP",
    "countrycode",
)
plt.xscale('log',base=2)
ax.set(xticks=[1/32,1/16,1/8,1/4,1/2,1,2], 
       xticklabels=["1/32","1/16","1/8","1/4","1/2","1","2"]);

In [ ]:
df

In [ ]:
(df.groupby("year")[["share_due_to_TFP","share_due_to_hcap","share_due_to_cap"]]
   .apply(lambda x: x.dropna().median())
   .plot(ylim=(0,1))
)